In [1]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer


In [2]:
from qiskit.test.mock import (
    FakeBogota,
    FakeCasablanca,
    FakeGuadalupe,
    FakeMontreal,
    FakeManhattan,
) 


In [ ]:
# Score generator
# for each MQTbench circuit
    # for each compiler -> tket, staq, qiskit, forest, ...
        # for each backend -> Rigetti, IBM, Google?, AQT?, OQC?, Cirq, ProjectQ
            # generate circuit
            # calculate score
            # safe data: input: qasmfile/gate count per gate, compiler/compiler settings/machine -> output: score

In [3]:
rigetti_m1 = {
    "provider": "rigetti",
    "name": "m1",
    "num_qubits": 80,
    "t1_avg": 33.845,
    "t2_avg": 28.230,
    "avg_gate_time_1q": 60e-3, #source: https://qcs.rigetti.com/qpus -> ASPEN-M-1
    "avg_gate_time_2q": 160e-3 #source: https://qcs.rigetti.com/qpus -> ASPEN-M-1
}
ibm_washington = {
    "provider": "ibm",
    "name": "washington",
    "num_qubits": 127,
    "t1_avg": 103.39,
    "t2_avg": 97.75,
    "avg_gate_time_1q": 206e-3, #estimated, based on the rigetti relation between 1q and 2q and given avg 2q time
    "avg_gate_time_2q": 550.41e-3 #source: https://quantum-computing.ibm.com/services?services=systems&system=ibm_washington
}
ionq = {
    "provider": "ionq",
    "name": "washington",
    "num_qubits": 127,
    "t1_avg": 10000,
    "t2_avg": 0.2,
    "avg_gate_time_1q": 0.00001,
    "avg_gate_time_2q": 0.0002 
}

In [34]:
def count_qubit_gates(qc, provider:str):
    dag = circuit_to_dag(qc)
    count_gates = dag.count_ops_longest_path()
    single_qubit_gates = 0
    two_qubit_gates = 0
    if provider == "ibm":
        # gates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
        if "id" in count_gates:
            single_qubit_gates += count_gates["id"]
        if "rz" in count_gates:
            single_qubit_gates += count_gates["rz"]
        if "sx" in count_gates:
            single_qubit_gates += count_gates["sx"]
        if "x" in count_gates:
            single_qubit_gates += count_gates["x"]
        if "cx" in count_gates:
            two_qubit_gates += count_gates["cx"]
        
    elif provider == "rigetti":
        # gates: rigetti_native_gates = ["rx", "rz", "cz"]
        if "rx" in count_gates:
            single_qubit_gates += count_gates["rx"]
        if "rz" in count_gates:
            single_qubit_gates += count_gates["rz"]
        if "cz" in count_gates:
            two_qubit_gates += count_gates["cz"]
            
    elif provider == "ionq":
        # gates: ionq_native_gates = ["ms", "rz", "ry", "rx"]
        if "rx" in count_gates:
            single_qubit_gates += count_gates["rx"]
        if "ry" in count_gates:
            single_qubit_gates += count_gates["ry"]
        if "rz" in count_gates:
            single_qubit_gates += count_gates["rz"]
        if "ms" in count_gates:
            two_qubit_gates += count_gates["ms"]
    return single_qubit_gates, two_qubit_gates
        

In [42]:
def calc_score_from_str(qc:str, backend):
    qc = QuantumCircuit.from_qasm_str(qc)
    return calc_score(qc, backend)
    
def calc_score_from_path(filepath, backend):       
    qc = QuantumCircuit.from_qasm_file(filepath)
    return calc_score(qc, backend)
    
def calc_score(qc:QuantumCircuit, backend):
    count_gates = count_qubit_gates(qc, backend["provider"])
    #print("Gates: ", count_gates)
    penalty_factor_width = 5000
    penalty_factor_1q = 500
    penalty_factor_2q = 1000

    b_qubits = backend["num_qubits"]
    t_1 = backend["t1_avg"]
    t_2 = backend["t2_avg"]
    avg_gate_time_1q = backend["avg_gate_time_1q"]
    avg_gate_time_2q = backend["avg_gate_time_2q"]
    max_depth_1q = min(t_1, t_2) / avg_gate_time_1q
    max_depth_2q = min(t_1, t_2) / avg_gate_time_2q

    penalty_width = 0
    
    if qc.num_qubits > b_qubits:
        penalty_width = penalty_factor_width
        
    
    score = count_gates[0] / max_depth_1q * penalty_factor_1q + count_gates[1] / max_depth_2q * penalty_factor_2q + penalty_width
    #print("Score: ", score)
    return score

# Qiskit

In [47]:
ibm_gates = ['id', 'rz', 'sx', 'x', 'cx', 'reset']
rigetti_gates = ["rx", "rz", "cz"]
ionq_gates = ["rxx", "rz", "ry", "rx"]

In [59]:
from qiskit import transpile
def get_qiskit_scores(qc_filepath:QuantumCircuit):
    qc = QuantumCircuit.from_qasm_file(qc_filepath)
    #compile to ibm architecture
    qc_ibm = transpile(qc, basis_gates=ibm_gates)
    score_ibm = calc_score_from_str(qc_ibm.qasm(), ibm_washington)
    #compile to rigetti architecture
    qc_rigetti = transpile(qc, basis_gates=rigetti_gates)
    score_rigetti = calc_score_from_str(qc_rigetti.qasm(), rigetti_m1)
    #compile to aqt architecture
    qc_ion = transpile(qc, basis_gates=ionq_gates)
    score_ionq = calc_score_from_str(qc_ion.qasm(), ionq)
    print(score_ibm, score_rigetti, score_ionq)

In [57]:
AQT_backend.configuration().basis_gates

['rx', 'ry', 'rxx']

In [56]:
from qiskit_aqt_provider import AQTProvider
aqt = AQTProvider("")
AQT_backend = aqt.backends.aqt_qasm_simulator

# Cirq

In [ ]:
import cirq
from cirq.contrib.svg import SVGCircuit

def get_cirq_compilation(qasm_qc, target):
    if "ibm" in target:
        pass
    elif "rigetti" in target:
        pass
    elif "ionq" in target:
        pass

In [ ]:
from cirq.contrib.qasm_import import circuit_from_qasm
test = open('../MQTbench/qasm_output/dj_indep_4.qasm', 'r').read()
ion_circuit = cirq.ion.ConvertToIonGates().convert_circuit(circuit)
print(ion_circuit)

In [ ]:
import cirq_rigetti
c = cirq_rigetti.circuit_transformers.default(circuit=circuit)
print(c)

SVGCircuit(c[0])

In [ ]:
import cirq
import cirq_ionq as ionq
q0 = cirq.LineQubit(0)
device = ionq.IonQAPIDevice([q0])
circuit = cirq.Circuit(device=device)
circuit.append(cirq.H(q0)**0.2) # Non-IonQ-API gate
print(circuit) # will print the circuit converted into IonQ's target gateset

# PyTKET

In [108]:
qc = qasm.circuit_from_qasm("../MQTbench/qasm_output/grover-noancilla_indep_2.qasm")
#compile to google gates
from pytket.circuit.display import render_circuit_jupyter
RebaseCirq().apply(qc)
render_circuit_jupyter(qc)

In [109]:
from pytket import qasm
from pytket.passes import RebaseCirq
from pytket.extensions.aqt import AQTBackend
from pytket.circuit.display import render_circuit_jupyter



def get_tket_scores(qc_filepath:QuantumCircuit):
    
    #aqt_backend = AQTBackend("MY_TOKEN", device_name="sim/noise-model-1")
    qc = qasm.circuit_from_qasm(qc_filepath)
    #compile to google gates
    RebaseCirq().apply(qc)
    render_circuit_jupyter(qc)
    qc_qasm = qasm.circuit_to_qasm_str(qc)
    score_ibm = calc_score_from_str(qc.qasm, rigetti_m1)

ImportError: cannot import name 'auto_rebase_pass' from 'pytket.passes' (/Users/nils/opt/anaconda3/envs/analyzer/lib/python3.9/site-packages/pytket/passes/__init__.py)

In [110]:
import os
i = 0
path_to_all_files = "../MQTbench/qasm_output"
res = dict()
for file in os.listdir(path_to_all_files):
    if "indep" in file:
        print(file, i)
        filepath = os.path.join(path_to_all_files, file)
        #qc = QuantumCircuit.from_qasm_file(filepath)
        get_qiskit_scores(filepath)
        get_tket_scores(filepath)
        i+=1
        if i>10:
            break

grover-noancilla_indep_2.qasm 0
3.161125319693095 3.188097768331562 0.05


QASMUnsupportedError: Gate of type U1q is not defined in header qelib1.inc

In [ ]:
from pytket.extensions.qiskit import IBMQBackend
from pytket import qasm
from pytket.passes import PlacementPass, RoutingPass, SequencePass
from pytket.routing import LinePlacement
from pytket.routing import GraphPlacement
      
backend = IBMQBackend("ibmq_quito")      
print(backend.required_predicates) 

c = qasm.circuit_from_qasm('../MQTbench/qasm_output/dj_indep_4.qasm')
render_circuit_jupyter(c)
compiled = backend.get_compiled_circuit(c)
render_circuit_jupyter(compiled)


PlacementPass(LinePlacement(backend.backend_info.architecture)).apply(compiled)
RoutingPass(backend.backend_info.architecture).apply(compiled)
render_circuit_jupyter(compiled)

#place = PlacementPass(GraphPlacement(b.backend_info.architecture)).apply(compiled)#PlacementPass(LinePlacement(b.backend_info.architecture))
#RoutingPass(backend.backend_info.architecture).apply(compiled)
#render_circuit_jupyter(compiled)
qasm_str = qasm.circuit_to_qasm_str(compiled)
print(calc_score_from_str(qasm_str, ibm_washington))

#from pytket.passes import RebaseCirq
#RebaseCirq().apply(c)
#render_circuit_jupyter(c)



In [ ]:
#pytket.routing.Architecture(coupling_map(connectivity)), 
connectivity = [[0,1], [1,2], [1,3], [3,4]] 
pytket.routing.Architecture(connectivity)
pytket.backends.Backend(name="test")

In [ ]:
c = qasm.circuit_from_qasm('../MQTbench/qasm_output/dj_indep_4.qasm')

In [ ]:
backend = IBMQBackend("ibmq_quito")  
backend.compile_circuit(c, optimisation_level=2)
render_circuit_jupyter(c)

qasm_str = qasm.circuit_to_qasm_str(c)
print(calc_score_from_str(qasm_str, ibm_washington))

#Qiskit Comparison
from qiskit import transpile
qiskit_qc = transpile(QuantumCircuit.from_qasm_file('../MQTbench/qasm_output/dj_indep_4.qasm'), basis_gates=['id', 'rz', 'sx', 'x', 'cx', 'reset'], optimization_level=1)
print(calc_score_from_str(qiskit_qc.qasm(), ibm_washington))

In [ ]:
from pytket.extensions.cirq import CirqStateSampleBackend
from pytket.extensions.pyquil import ForestBackend
from pytket.extensions.ionq import IonQBackend
backend = IonQBackend()
backend.compile_circuit(c)
render_circuit_jupyter(c)

In [ ]:
from pyquil import get_qc
from pytket.passes import RebaseQuil
RebaseQuil().apply(c)
render_circuit_jupyter(c)

In [ ]:
backend = CirqStateSampleBackend()
backend.compile_circuit(c)
render_circuit_jupyter(c)

# Pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [ ]:
ibm_native_gates = FakeMontreal().configuration().basis_gates
rigetti_native_gates = ["rx", "rz", "cz"]
ibm_native_gates

dev = qml.device('default.qubit', wires=[0, 1, 2])
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)(0.2, 0.3, 0.4))

compiled_qfunc = qml.compile(basis_set=rigetti_native_gates)(qfunc)
compiled_qnode = qml.QNode(compiled_qfunc, dev)
print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4))

In [ ]:
path_file = "../MQTbench/qasm_output/dj_indep_4.qasm"
with open (path_file, "r") as myfile:
    data = myfile.read()
my_circuit = qml.from_qasm(data)

n=4
dev = qml.device('default.qubit', wires=n)
@qml.qnode(dev)
def circuit():
    my_circuit(wires=tuple(range(n)))
    return [qml.expval(qml.PauliZ(i)) for i in range(n)]

#circuit()
#print(dev._circuit.qasm(formatted=True))

In [ ]:
pennylane_ibm=["RZ", "SX", "PauliX", "CNOT"]
pennylane_rigetti=["RX", "RZ", "CZ"]

In [ ]:
compiled_qfunc1 = qml.compile()(circuit)
compiled_qnode1 = qml.QNode(compiled_qfunc1, dev)
print(qml.draw(compiled_qnode1)())

compiled_qfunc2 = qml.compile(
    basis_set=["CNOT", "RX", "RY", "RZ"],
    num_passes=2
)(circuit)

compiled_qnode2 = qml.QNode(compiled_qfunc2, dev)
print(qml.draw(compiled_qnode2)())

In [ ]:
def qfunc(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.PauliY(wires=2)
    qml.CY(wires=[1, 2])
    return qml.expval(qml.PauliZ(wires=0))

dev = qml.device('default.qubit', wires=[0, 1, 2])
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)(0.2, 0.3, 0.4))

compiled_qfunc = qml.compile(
    pipeline=[
        qml.transforms.commute_controlled(direction="left"),
        qml.transforms.merge_rotations(atol=1e-6),
        qml.transforms.cancel_inverses
    ],
    basis_set=["CNOT", "RX", "RY", "CZ"],
    num_passes=2
)(qfunc)

compiled_qnode = qml.QNode(compiled_qfunc, dev)
print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4))

In [ ]:
def qfunc():
    qml.Hadamard(wires=0)    
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(-0.4, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(-0.4, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(-0.4, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-0.4, wires=2)
    qml.RX(1, wires=2)
    qml.PauliY(wires=2)
    qml.CY(wires=[1, 2])
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]

dev = qml.device('default.qubit', wires=[0, 1, 2])
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)())

compiled_qfunc = qml.compile(
    basis_set=["CNOT", "Rdawd", "wdfawdaw", "dawd"],
    num_passes=20
)(qfunc)

compiled_qnode = qml.QNode(compiled_qfunc, dev)
print(qml.draw(compiled_qnode)())

# ProjectQ

In [ ]:

import projectq.setups.ibm as ibm_setup

# Microsoft QDK

# Old

In [43]:
import os
i = 0
path_to_all_files = "../MQTbench/qasm_output"
res = dict()
for file in os.listdir(path_to_all_files):
    print (file)
    if "ibm" in file:
        score = calc_score_from_path(os.path.join(path_to_all_files, file), ibm_washington)
        res[file] = score
    elif "rigetti" in file:
        score= calc_score_from_path(os.path.join(path_to_all_files, file), rigetti_m1)
        res[file] = score
    i+=1
    if i>50:
        break

grover-noancilla_nativegates_ibm_opt3_2.qasm
grover-v-chain_nativegates_ibm_opt2_3.qasm
grover-noancilla_mapped_ibm-b_opt3_5.qasm
qftentangled_mapped_rigetti-b_opt1_8.qasm
dj_nativegates_ibm_opt2_3.qasm
grover-v-chain_mapped_ibm-s_opt1_2.qasm
grover-noancilla_mapped_rigetti-b_opt0_3.qasm
grover-v-chain_nativegates_rigetti_opt1_3.qasm
grover-noancilla_indep_2.qasm
qpeinexact_nativegates_ibm_opt1_8.qasm
grover-noancilla_nativegates_rigetti_opt1_4.qasm
grover-v-chain_mapped_rigetti-b_opt2_4.qasm
dj_mapped_rigetti-s_opt0_3.qasm
dj_indep_4.qasm
wstate_nativegates_rigetti_opt1_8.qasm
dj_nativegates_ibm_opt0_2.qasm
grover-v-chain_nativegates_ibm_opt0_2.qasm
grover-noancilla_mapped_ibm-b_opt1_4.qasm
hhl_mapped_ibm-b_opt0_5.qasm
grover-noancilla_nativegates_ibm_opt1_3.qasm
grover-noancilla_mapped_rigetti-b_opt2_2.qasm
hhl_mapped_rigetti-s_opt2_8.qasm
hhl_nativegates_ibm_opt2_5.qasm
grover_nativegates_rigetti_opt1_5.qasm
hhl_nativegates_rigetti_opt2_8.qasm
grover-v-chain_mapped_ibm-s_opt3_3.qasm